In [1]:
import pandas as pd
import numpy as np
import os
import math

获取所需要的标签

In [1]:
columns_ls = ['Description',
 'Qualitative Measure',
 'Measurement Inequality',
 'Quantitative measurement',
 'Units',
 'Allele Name',
 'MHC allele class',
 'Type',
 'Date',
 'Title',
 'Method/Technique',
 'Assay Group',
 'Name']

new_columns_ls = ['Description',
 'Qualitative Measure',
 'Measurement Inequality',
 'Quantitative measurement',
 'Units',
 'Allele Name',
 'MHC allele class',
 'Ref Type',
 'Ref Date',
 'Ref Title',
 'Assay Method/Technique',
 'Assay Group',
 'Host Name']

 # 输入与输出路径
IEDB_origin = '/raid/hly/PK-panAllele/data/mhc_ligand_full.csv' 
IEDB_ori_need = '/raid/hly/PK-panAllele/data/IEDB2022_need.csv'

In [25]:
df = pd.read_csv(IEDB_origin,header=1)

/raid/hly/CODE/miniconda3/envs/vaccin/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (5,14,15,16,17,18,19,20,21,22,23,24,25,26,29,30,31,32,33,34,35,36,37,38,41,42,43,44,45,46,47,48,49,50,53,54,55,56,57,58,59,60,61,62,63,64,65,68,69,70,71,72,73,74,75,76,77,78,81,84,89,90,91,92,93,94,97,99) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [28]:
# 获取原始文件中需要的标签。存至“IEDB_ori_need_Data”
df_need = df[columns_ls]
df_need.columns = [new_columns_ls]
df_need.to_csv(IEDB_ori_need,index=0)

删除不需要的数据

In [ ]:
#读取文件
df_all = pd.read_csv('/raid/hly/PK-panAllele/data/IEDB2022_need.csv')

In [5]:
# 筛选出host为人类的条目--Homo sapiens (human) or human（Homo sapiens）
df_host = df_all[(df_all['Host Name'] == 'Homo sapiens (human)') | (df_all['Host Name'] =='human (Homo sapiens)') ]

# 筛选出有measurement标签的数值
df_QM = df_host[pd.notnull(df_host['Quantitative measurement'])]

#保留规范化的多肽序列（不含符号且不含'BXJZ'）
    # 删除seq含'-'或' '的行
df1 = df_QM.drop(index=(df_QM.loc[(df_QM['Description'].str.contains('-'))].index))
df2 = df1.drop(index=(df1.loc[(df1['Description'].str.contains(' '))].index))
    # 删除seq含'BXJZ'的行
df3 = df2.drop(index=(df2.loc[(df2['Description'].str.contains('B'))].index))
df4 = df3.drop(index=(df3.loc[(df3['Description'].str.contains('X'))].index))
df5 = df4.drop(index=(df4.loc[(df4['Description'].str.contains('J'))].index))
df6 = df5.drop(index=(df5.loc[(df5['Description'].str.contains('Z'))].index))

# 筛选出精确的allele name（含'HLA'，'：'） 
df_QM = df6.loc[(df6['Allele Name'].str.contains('HLA'))]
df_QM = df_QM.loc[(df_QM['Allele Name'].str.contains(":"))]

# 删去assay group 不为 “KD”及 “IC50”，“EC50”相关的条目
    #删去 assay group为 3D structure 的条目
df_assay_1 = df_QM.drop(index=(df_QM.loc[(df_QM['Assay Group']== '3D structure' )].index))
    #删去 assay group为 50% dissociation temperature 的条目
df_assay_2 = df_assay_1.drop(index=(df_assay_1.loc[(df_assay_1['Assay Group']== '50% dissociation temperature' )].index))
    #删去 assay group为 half life 的条目
df_assay_3 = df_assay_2.drop(index=(df_assay_2.loc[(df_assay_2['Assay Group']== 'half life' )].index))
     #删去 assay group为 off rate 的条目
df_assay_4 = df_assay_3.drop(index=(df_assay_3.loc[(df_assay_3['Assay Group']== 'off rate' )].index))
    #删去 assay group为 on rate 的条目
df_assay_5 = df_assay_4.drop(index=(df_assay_4.loc[(df_assay_4['Assay Group']== 'on rate' )].index))
    #删去 assay group为 qualitative binding 的条目
df_assay_6 = df_assay_5.drop(index=(df_assay_5.loc[(df_assay_5['Assay Group']== 'qualitative binding' )].index))

# 删去'Measurement Inequality'为'>','<','>=','<='的行
df_MI_1 = df_assay_6.drop(index=(df_assay_6.loc[(df_assay_6['Measurement Inequality']== '>' )].index))
df_MI_2 = df_MI_1.drop(index=(df_MI_1.loc[(df_MI_1['Measurement Inequality']== '<' )].index))
df_MI_3 = df_MI_2.drop(index=(df_MI_2.loc[(df_MI_2['Measurement Inequality']== '>=' )].index))
df_MI_4 = df_MI_3.drop(index=(df_MI_3.loc[(df_MI_3['Measurement Inequality']== '<=' )].index))

# 去除活性值大于50000的条目
df_QM = df_MI_4.drop(index=(df_MI_4.loc[(df_MI_4['Quantitative measurement'] > 50000)].index))

# 去除“reference Title”为“Quantitating T cell cross-reactivity for unrelated peptide antigens.”的条目。
df_QM = df_QM.drop(index=(df_QM.loc[(df_QM['Ref Title']== 'Quantitating T cell cross-reactivity for unrelated peptide antigens.' )].index))


In [59]:
# 数据标准化 【1-log10(IC50)/log10(50000 nM)】 ,IC50单位为nM
df_QM["Normalized_QM"] = 1-(df_QM["Quantitative measurement"].apply(np.log10))/np.log10(50000)

In [60]:
# 重置索引并保存文件
df_QM = df_QM.reset_index(drop = True)

df_QM.to_csv('/raid/hly/PK-panAllele/data/IEDB2022_processed.csv') 

获得测试集--取出IEDB2022中 "Ref Date >= 2021"的条目

In [7]:
df_QM = pd.read_csv('/raid/hly/PK-panAllele/data/IEDB2022_processed.csv',index_col=0)

In [14]:
df_2021 = df_QM[df_QM['Ref Date'] >= 2021]
df_2021.to_csv('/raid/hly/PK-panAllele/data/IEDB2021+2022.csv')

In [5]:
out_dir = '/raid/hly/PK-panAllele/data/test_data'
for name, group in df_2021.groupby('Allele Name') :
    # 替换符号，如将/替换成&
    name = name.replace('/', '&')
    name = name.replace('*','_')
    name = name.replace(':','')
    pd.DataFrame(group).to_csv(os.path.join(out_dir,name+'.csv'))

In [21]:
# IEDB 2020年及之前的数据
df_before_2021 = df_QM[df_QM['Ref Date'] < 2021]
df_before_2021.to_csv('/raid/hly/PK-panAllele/data/IEDB_before_2021.csv')

处理“BD2013”数据集

In [42]:
# 读取 txt 文件
ls_out=[]
 
#打开文件
with open('/raid/hly/PK-panAllele/data/BD2013.txt','r')as df_in:
    #读每一行
    for line in df_in:
        list = line.strip().split('\t')    #".strip()",清除前后空格
        # seq = [''.join(list[:-2])]         #将多个氨基酸字符元素合并为一个元素
        ls_out.append(list)
df_in.close()

In [43]:
df_BD2013 = pd.DataFrame(ls_out[1:],columns=ls_out[0])

In [63]:
# 处理数据
# 筛选出species为人类的条目
df_host = df_BD2013[(df_BD2013['species'] == 'human')]

# 筛选出有meas数值的条目
df_QM = df_host[pd.notnull(df_host['meas'])]

#保留规范化的多肽序列（不含符号且不含'BXJZ'）
    # 删除seq含'-'或' '的行
df1 = df_QM.drop(index=(df_QM.loc[(df_QM['sequence'].str.contains('-'))].index))
df2 = df1.drop(index=(df1.loc[(df1['sequence'].str.contains(' '))].index))
    # 删除seq含'BXJZ'的行
df3 = df2.drop(index=(df2.loc[(df2['sequence'].str.contains('B'))].index))
df4 = df3.drop(index=(df3.loc[(df3['sequence'].str.contains('X'))].index))
df5 = df4.drop(index=(df4.loc[(df4['sequence'].str.contains('J'))].index))
df6 = df5.drop(index=(df5.loc[(df5['sequence'].str.contains('Z'))].index))

# 筛选出精确的allele name（含'HLA'，'：'） 
df_QM = df6.loc[(df6['mhc'].str.contains('HLA'))]
df_QM = df_QM.loc[(df_QM['mhc'].str.contains(":"))]

# 仅保留'inequality'为'='的行
df_MI = df_QM[(df_QM['inequality']=='=')]

# 去除活性值大于50000的条目
df_BD2013 = df_MI.drop(index=(df_MI.loc[(df_MI['meas'].astype(float) > 50000)].index))

In [66]:
# 数据标准化 【1-log10(IC50)/log10(50000 nM)】 ,IC50单位为nM
df_BD2013["Normalized_QM"] = 1-(df_BD2013["meas"].astype(float).apply(np.log10))/np.log10(50000)

In [72]:
# 重置索引并保存文件
df_BD2013 = df_BD2013.reset_index(drop = True)

df_BD2013.to_csv('/raid/hly/PK-panAllele/data/BD2013_processed.csv') 

获得训练集（IEDBbefore2021 + BD2013）

In [90]:
columns_ls_1 = ['Description',
 'Quantitative measurement',
 'Normalized_QM',
 'Allele Name',
 'MHC allele class',
 ]

columns_ls_2 = ['sequence',
 'meas',
 'Normalized_QM',
 'mhc',
 'MHC allele class',
 ]


In [82]:
df1 = pd.read_csv('/raid/hly/PK-panAllele/data/IEDB_before_2021.csv',index_col=0)
df1 = df1[columns_ls_1 ]

In [83]:
df2 = pd.read_csv('/raid/hly/PK-panAllele/data/BD2013_processed.csv',index_col=0)
df2['MHC allele class'] = 'I'
df2 = df2[columns_ls_2]
df2.columns = columns_ls_1

In [117]:
# 拼接IEDBbefore2021 + BD2013文件
df3 = pd.concat([df1,df2])

In [119]:
train_dir = '/raid/hly/PK-panAllele/data/train_MID'
for name, group in df3.groupby('Allele Name') :
    # 替换符号，如将/替换成&
    name = name.replace('/', '&')
    name = name.replace('*','_')
    name = name.replace(':','')
    pd.DataFrame(group).to_csv(os.path.join(train_dir,name+'.csv'))

In [2]:
train_dir = '/raid/hly/PK-panAllele/data/train_MID'
train_dir_1 = '/raid/hly/PK-panAllele/data/train_MID_1'
train_dir_2 = '/raid/hly/PK-panAllele/data/train_MID_2'
test_dir = '/raid/hly/PK-panAllele/data/test_data'

for file in os.listdir(train_dir):
    if os.path.exists(os.path.join(test_dir,file)) :
        out_file = train_dir_1  
    else:
        out_file = train_dir_2

    df = pd.read_csv(os.path.join(train_dir,file),index_col=0)

#### 去重,取中值
    #按 Description列排序
    df.sort_values(by='Description',axis=0,ascending='True', inplace=True)

    #重置索引
    df = df.reset_index(drop = True)

    #创建新表，用来保存有重复的Description及其对应的measurement中位数
    df_uniq = pd.DataFrame(columns = ['Description','Normalized_QM'])

    #获得所有重复的 Description及其 measurement的中位数
    j = 1

    # Do not use parallel!!使用并行会出错！
    for i in range(0, len(df)):
        if(i == 0 or df['Description'][i] != df['Description'][i-1]):
            #取出所有Description==df['Description'][i]的行
            df_tmp = df[df['Description'] == df['Description'][i]]
            
            if(len(df_tmp) > 0):
                df_tmp.sort_values(by='Normalized_QM',axis=0,ascending='True', inplace=True)

                df_tmp = df_tmp.reset_index(drop = True)

                index = math.ceil(len(df_tmp) / 2) - 1
                measurement = df_tmp['Normalized_QM'][index]
                df_uniq = df_uniq.append({'Description':df['Description'][i],'Normalized_QM':measurement}, ignore_index=True)

    df = df.drop_duplicates(subset = ['Description'], keep = 'first')
    df = df.reset_index(drop = True)

    # 重新确认。在df中找到与df_unique相同的Description和对应Quantitative measurement值
    for i in range(0, len(df_uniq)):
        df.loc[df['Description'] == df_uniq['Description'][i], ['Normalized_QM']] = df_uniq['Normalized_QM'][i]

    df.to_csv(os.path.join(out_file,file),index=0)  


/raid/hly/CODE/miniconda3/envs/vaccin/lib/python3.6/site-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [6]:
# 统计

#统计了各数据集的条目数，多肽长度的最值，活性值最值，标准化活性值的最值。

files_dir = '/raid/hly/PK-panAllele/data/train_MID_2'
files = os.listdir(files_dir)
statistic = []
for file in files:
    df = pd.read_csv(os.path.join(files_dir,file))
    file_name = file.split('.')[0]
    # 条目数
    pep_num = len(df)

    #多肽长度范围
    pep_len_min = min(map(len,df['Description']))
    pep_len_max = max(map(len,df['Description']))

    #QM范围
    QM_min = min(df['Quantitative measurement'])
    QM_max = max(df['Quantitative measurement'])
    
    #log_QM范围
    Normalized_QM_min = min(df['Normalized_QM'])
    Normalized_QM_max = max(df['Normalized_QM'])

    statistic.append([file_name,pep_num,pep_len_min,pep_len_max,QM_min,QM_max,Normalized_QM_min,Normalized_QM_max])
df_stt = pd.DataFrame(statistic,columns=['HLA-allele','pep_num','pep_len_min','pep_len_max','QM_min','QM_max','Normalized_QM_min','Normalized_QM_max'])
df_stt.sort_values(by='pep_num',axis=0,ascending=False, inplace=True)
df_stt = df_stt.reset_index(drop = True)
df_stt.to_csv('/raid/hly/PK-panAllele/data/train_MID_2_statistics220708.csv')


In [2]:
dft = pd.read_csv('/raid/hly/PK-panAllele/data/IEDB2021+2022.csv',index_col=0)

In [ ]:
dft = dft[(dft['MHC allele class']=='II')]

In [8]:
j = 0
for s in dft['Description']:
    if len(s)<13 or len(s)>21:
        j = j + 1

In [9]:
j

52